# Team Analytics Alliance

## Covid-19 and it's impact on the Airline Industry

In [1]:
# Import Dependencies and Setup
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

## Add, load and read CSV files

In [2]:
# Add CSV files
# File to Load (Remember to Change These)
airline_flights_2019_to_load = "Resources/OnewayT_SCHEDULE_T32019.csv"
airline_flights_2020_to_load = "Resources/OnewayT_SCHEDULE_T32020.csv"
tsa_file = "Resources/TSA_checkpoint.csv"
csv_path_1 = "AverageFare_Q2_2019.csv"
csv_path_2 = "AverageFare_Q2_2020.csv"

# Read School and Student Data File and store into Pandas DataFrames
airline_flights_2019 = pd.read_csv(airline_flights_2019_to_load)
airline_flights_2020 = pd.read_csv(airline_flights_2020_to_load)
tsa_data = pd.read_csv(tsa_file, thousands=',')
revenue = pd.read_csv("Resources/Yearly_Income.csv")
df_2019 = pd.read_csv(csv_path_1)
df_2020 = pd.read_csv(csv_path_2)

FileNotFoundError: [Errno 2] File AverageFare_Q2_2019.csv does not exist: 'AverageFare_Q2_2019.csv'

## Clean our CSVs

In [ ]:
# Change the name of the columnn to say 2019 Total Number of Flights from the Airline Flights 2019 DataFrame.
airline_flights_19 = airline_flights_2019.rename(columns={"Description": "Airline Company", "Sum(REV_ACRFT_DEP_PERF_510)": "2019 Total Number of Flights"})


# Change the name of the columnn to say 2020 Total Number of Flights from the US Airline Flights 2020 DataFrame.
airline_flights_20 = airline_flights_2020.rename(columns={"Description": "Airline Company", "Sum(REV_ACRFT_DEP_PERF_510)": "2020 Total Number of Flights"})

# Drop the columns we don't need.
clean_airline_flights_19_20 = airline_flights_19_20.drop(['Airline Company_y', 'UNIQUE_CARRIER'], axis=1)

#Rename the columns
tsa_df = tsa_data.rename(columns={"Total Traveler Throughput":"Traveler Throughput 2020", "Total Traveler Throughput (1 Year Ago - Same Weekday)":"Traveler Throughput 2019"})

#Convert the "Date" column to Pandas yymmdd format
tsa_df["Date"] = pd.to_datetime(tsa_df["Date"])

#Extract just the month and date from the "Date" column 
tsa_df['Month/Date'] = tsa_df['Date'].dt.strftime('%m-%d')

#The columns are the wrong data type. Convert the throughput columns from object to numeric.
tsa_df["Traveler Throughput 2019"] = pd.to_numeric(tsa_df["Traveler Throughput 2019"])
tsa_df["Traveler Throughput 2020"] = pd.to_numeric(tsa_df["Traveler Throughput 2020"])

#Delete the Date column and reorder the columns
# del tsa_df["Date"]
tsa_df = tsa_df [["Date", "Month/Date", "Traveler Throughput 2019", "Traveler Throughput 2020"]]

# Merging our CSVs

In [ ]:
# Combine the data into a single dataset so we can have 2019 and 2020 flight on one DataFrame.  
airline_flights_19_20 = pd.merge(airline_flights_19, airline_flights_20, how="left", on=["UNIQUE_CARRIER", "UNIQUE_CARRIER"])

In [ ]:
# Fill in columns that have Nan to 0 to indicate 0 flights took place.
caf_19_20 = clean_airline_flights_19_20.fillna(0)

In [ ]:
# Change the column name to reflect US Airline Companies. This will show the names of the Airlines that flew over 2019 and 2020.
cafs_19_20 = caf_19_20.rename(columns={"Airline Company_x": "Airline Companies"})

In [ ]:
# Find the sum of the total number of US flights taken in 2019, 2020 and the percentage change from both years.
total_number_of_flights_2019 = cafs_19_20['2019 Total Number of Flights'].sum()
total_number_of_flights_2020 = cafs_19_20['2020 Total Number of Flights'].sum()
percentage_change_19_20 = ((total_number_of_flights_2020 - total_number_of_flights_2019)/total_number_of_flights_2019)*100

print(f"The total number of flights taken in 2019 is {total_number_of_flights_2019}.")
print(f"The total number of flights taken in 2019 is {total_number_of_flights_2020}.")
print(f"The change in the number of flights from top airlines from 2019 to 2020 is {percentage_change_19_20.round()}%.")

In [ ]:
# Set the Airline Companies column as the DataFrame's index.
CAF_19_20 = cafs_19_20.set_index('Airline Companies')

In [ ]:
# Sort the US Airline 2019-2020 Flights in order from the most amount of flights taken to the least.
tops_airline_flights_19_20 = CAF_19_20.sort_values(by='2019 Total Number of Flights', ascending=False)

In [ ]:
# Display only airlines that had over 130,000 flights from 2019.
top_airline_flights_19_20 = tops_airline_flights_19_20.loc[(tops_airline_flights_19_20['2019 Total Number of Flights']>130000)]

In [ ]:
# Find the percentage change in flights between the years 2019 and 2020.
percentage_change = ((top_airline_flights_19_20['2020 Total Number of Flights']-top_airline_flights_19_20['2019 Total Number of Flights'])/top_airline_flights_19_20['2019 Total Number of Flights']*100)

# Creating bar graphs to compare US flight data between 2019 and 2020.

In [ ]:
# Create a bar graph to show the percentage change in flights.
pc = percentage_change.plot.bar(rot=90, figsize=(20,10))

pc.set_ylabel("Percentage Change")
pc.set_title('Percentage Change in Flights between 2019 & 2020')

plt.savefig('output_data/Percentage_Change_in_Flights_between_2019_and_2020.png')
plt.show()

In [ ]:
# Create a bar graph that shows the the number of flights taken from the top US Airline Companies in 2019 and 2020.
taf = top_airline_flights_19_20.plot.bar(rot=90, figsize=(20,10))
taf.set_ylabel("Number of Flights (Millions)")
taf.set_title('Airlines with the most Flights from 2019 and 2020')
plt.savefig('output_data/Top_Airline_Flights_19_20.png')
plt.show()